In [ ]:
# PROYECTO FINAL - PROGRAMACIÓN AVANZADA
# Predicción de indicadores económicos de Colombia

# Importación de librerías necesarias
import pandas as pd                  # Para manejar y analizar datos (dataframes)
import numpy as np                   # Para realizar operaciones numéricas y manejo de arreglos
import plotly.graph_objects as go    # Para visualizar datos (gráficas interactivas)
import requests                      # Para conectarse a la API

# Librerías de machine learning (scikit-learn)
from sklearn.ensemble import RandomForestRegressor # Modelo de bosque aleatorio para regresión
from sklearn.metrics import mean_squared_error, mean_absolute_error # Métricas de evaluación

# Descarga de datos económicos desde Github
url = 'https://raw.githubusercontent.com/Wiky-25/Laelegida/refs/heads/main/Datos_dict.json'
r = requests.get(url)

if r.status_code == 200:
    print('Acceso exitoso a la base de datos\n')
    response_dict = r.json()
else:
    print(f'No se pudo acceder a los datos, código de error: {r.status_code}')

df = pd.DataFrame(response_dict).T   # Creamos DataFrame y transponemos

# Limpieza y conversión de datos
df['Tasa de política monetaria(Dato fin de año)'] = df['Tasa de política monetaria(Dato fin de año)'].str.replace(',','.').astype(float)
df['Inflación total, anual(Dato fin de año)'] = df['Inflación total, anual(Dato fin de año)'].str.replace(',','.').astype(float)
df['Inflación sin alimentos ni regulados, anual(Dato fin de año)'] = df['Inflación sin alimentos ni regulados, anual(Dato fin de año)'].str.replace(',','.').astype(float)
df['Salario mínimo mensual, variación anual(Dato fin de año)'] = df['Salario mínimo mensual, variación anual(Dato fin de año)'].str.replace(',','.').astype(float)

# TRM: eliminar punto de miles, cambiar coma por punto decimal
df['Tasa Representativa del Mercado (TRM)(Dato fin de año)'] = df['Tasa Representativa del Mercado (TRM)(Dato fin de año)'].str.replace('.', '')
df['Tasa Representativa del Mercado (TRM)(Dato fin de año)'] = df['Tasa Representativa del Mercado (TRM)(Dato fin de año)'].str.replace(',','.').astype(float)

# Conversión de Fecha a Año
df['Año'] = pd.to_datetime(df['Fecha'], dayfirst=True).dt.year
df['Año'] = df['Año'].astype(int)

# Limpieza de nombres de columnas
df.columns = [col.replace('(Dato fin de año)', '').strip() for col in df.columns]

df = df.rename(columns={
    "Tasa Representativa del Mercado (TRM)": "Tasa Representativa del Mercado COP/USD",
    "Tasa de política monetaria": "Tasa de política monetaria (%)",
    "Inflación total, anual": "Inflación total anual (%)",
    "Inflación sin alimentos ni regulados, anual": "Inflación sin alimentos ni regulados anual (%)",
    "Salario mínimo mensual, variación anual": "Salario mínimo mensual, variación anual (%)"
})

# Lista de índices disponibles
indices = [col for col in df.columns if col.lower() not in ['fecha', 'año']]

# Clase Pataconcito con métricas y gráficos
class Pataconcito:
    """Clase que maneja la interacción con el usuario para predecir
    indicadores económicos usando el modelo 'Pataconcito'."""

    def __init__(self, df, indices):
        self.df = df
        self.indices = indices

    def interfaz(self):
        while True:
            print('Estos son los índices disponibles a predecir:\n')
            for i, indice_nombre in enumerate(self.indices):
                print(f'{i+1}. {indice_nombre.capitalize()}')

            indice_input = input("\nIngrese el nombre del índice que desea predecir o escriba 'salir' para terminar: ").strip().lower()
            indices_lower = [x.strip().lower() for x in self.indices]

            if indice_input == "salir":
                print("El programa ha finalizado")
                break

            if indice_input in indices_lower:
                indice = self.indices[indices_lower.index(indice_input)]
                print(f"\nBien, Has seleccionado: {indice}")

                y_real = self.df[indice].values
                X_full = self.df.drop(columns=[indice, 'Año', 'Fecha'])
                anios = self.df['Año'].values

                predicciones = []
                rmse_lista = []
                mae_lista = []

                for i in range(len(self.df)):
                    X_train = X_full.iloc[:i]
                    y_train = y_real[:i]
                    X_test = X_full.iloc[[i]]
                    y_test = y_real[i:i+1]

                    if len(X_train) < 3:
                        predicciones.append(np.nan)
                        rmse_lista.append(np.nan)
                        mae_lista.append(np.nan)
                        continue

                    modelo_temp = RandomForestRegressor(n_estimators=100, random_state=42)
                    modelo_temp.fit(X_train, y_train)
                    pred = modelo_temp.predict(X_test)[0]
                    predicciones.append(pred)

                    # Métricas por año
                    rmse = np.sqrt(mean_squared_error(y_test, [pred]))
                    mae = mean_absolute_error(y_test, [pred])
                    rmse_lista.append(rmse)
                    mae_lista.append(mae)

                self.df['Predicción_pataconcito'] = predicciones
                self.df['RMSE'] = rmse_lista
                self.df['MAE'] = mae_lista

                # Métricas promedio
                rmse_prom = np.nanmean(rmse_lista)
                mae_prom = np.nanmean(mae_lista)
                print(f"\nRMSE promedio: {rmse_prom:.3f}")
                print(f"MAE promedio: {mae_prom:.3f}")

                # Gráfica interactiva mejorada
                fig = go.Figure()

                # Valor real
                fig.add_trace(go.Scatter(
                    x=self.df['Año'],
                    y=self.df[indice],
                    mode='lines+markers',
                    name='Valor real',
                    line=dict(color='#ab87ff', width=4),
                    marker=dict(size=8, symbol='circle')
                ))

                # Predicción
                fig.add_trace(go.Scatter(
                    x=self.df['Año'],
                    y=self.df['Predicción_pataconcito'],
                    mode='lines+markers',
                    name='Predicción Pataconcito',
                    line=dict(color='#c1ff9b', width=3, dash='dot'),
                    marker=dict(size=8, symbol='diamond')
                ))

                # Layout
                fig.update_layout(
                    title=f"Backtesting del modelo 'Pataconcito' para {indice}",
                    xaxis_title='Año',
                    yaxis_title=indice,
                    template='plotly_dark',
                    hovermode='x unified',
                    legend=dict(x=0.01, y=0.99, bgcolor='rgba(0,0,0,0)'),
                    font=dict(family="Arial", size=14)
                )

                fig.show()

            else:
                print("Error: índice no válido. Intente de nuevo.\n")

if __name__ == "__main__":
  Proyecto = Pataconcito(df, indices)
  Proyecto.interfaz()


Acceso exitoso a la base de datos

Estos son los índices disponibles a predecir:

1. Tasa representativa del mercado cop/usd
2. Tasa de política monetaria (%)
3. Inflación total anual (%)
4. Inflación sin alimentos ni regulados anual (%)
5. Salario mínimo mensual, variación anual (%)

Ingrese el nombre del índice que desea predecir o escriba 'salir' para terminar: Tasa representativa del mercado cop/usd

Bien, Has seleccionado: Tasa Representativa del Mercado COP/USD

RMSE promedio: 614.263
MAE promedio: 614.263


Estos son los índices disponibles a predecir:

1. Tasa representativa del mercado cop/usd
2. Tasa de política monetaria (%)
3. Inflación total anual (%)
4. Inflación sin alimentos ni regulados anual (%)
5. Salario mínimo mensual, variación anual (%)

Ingrese el nombre del índice que desea predecir o escriba 'salir' para terminar: salir
El programa ha finalizado
